In [13]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from torchvision.datasets import CocoDetection
from PIL import Image
from tqdm import tqdm
import os

# Dataset class for COCO
class COCODataset(CocoDetection):
    def __init__(self, root, annFile, transforms=None):
        super(COCODataset, self).__init__(root, annFile)
        self.transforms = transforms

    def __getitem__(self, idx):
        img, target = super(COCODataset, self).__getitem__(idx)
        if self.transforms:
            img, target = self.transforms(img, target)

        boxes = []
        labels = []
        for obj in target:
            # Convert [x_min, y_min, width, height] to [x_min, y_min, x_max, y_max]
            bbox = obj["bbox"]
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height

            if width > 0 and height > 0:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(obj["category_id"])

        if len(boxes) == 0:
            return self.__getitem__((idx + 1) % len(self))

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64)
        }

        return img, target


def custom_transforms(img, target):
    if not isinstance(img, torch.Tensor):
        img = F.to_tensor(img)
    return img, target


def get_coco_data_loader(root, annFile, num_images=None, batch_size=4):
    dataset = COCODataset(root, annFile, transforms=custom_transforms)

    if num_images:
        indices = list(range(min(num_images, len(dataset))))
        dataset = Subset(dataset, indices)

    return DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


def compute_iou(pred_boxes, true_boxes):
    inter_xmin = torch.max(pred_boxes[:, None, 0], true_boxes[:, 0])
    inter_ymin = torch.max(pred_boxes[:, None, 1], true_boxes[:, 1])
    inter_xmax = torch.min(pred_boxes[:, None, 2], true_boxes[:, 2])
    inter_ymax = torch.min(pred_boxes[:, None, 3], true_boxes[:, 3])

    inter_area = (inter_xmax - inter_xmin).clamp(0) * (inter_ymax - inter_ymin).clamp(0)
    pred_area = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
    true_area = (true_boxes[:, 2] - true_boxes[:, 0]) * (true_boxes[:, 3] - true_boxes[:, 1])

    union_area = pred_area[:, None] + true_area - inter_area
    iou = inter_area / union_area.clamp(min=1e-6)
    return iou


def evaluate(model, data_loader, device):
    model.eval()
    all_iou = []
    total_predictions = 0
    correct_predictions = 0

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluation Loop"):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            outputs = model(images)

            for i, output in enumerate(outputs):
                pred_boxes = output['boxes']
                true_boxes = targets[i]['boxes']

                
                if pred_boxes.size(0) == 0 or true_boxes.size(0) == 0:
                    continue

                
                iou = compute_iou(pred_boxes, true_boxes)
                max_iou, indices = iou.max(dim=1) 
                matched_labels = targets[i]['labels'][indices]

                all_iou.append(max_iou.mean().item())

                
                pred_labels = output['labels']
                total_predictions += len(pred_labels)
                correct_predictions += (pred_labels == matched_labels).sum().item()

    mean_iou = sum(all_iou) / len(all_iou) if all_iou else 0
    mAP = correct_predictions / total_predictions if total_predictions > 0 else 0
    wada = (mean_iou + mAP) / 2 

    return mAP, wada, mean_iou

def train(model, train_loader, optimizer, device):
    model.train()
    for images, targets in tqdm(train_loader, desc="Training Loop"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = fasterrcnn_resnet50_fpn(pretrained=True).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    dataset_root = "/kaggle/input/coco-2017-dataset/coco2017/"
    train_loader = get_coco_data_loader(os.path.join(dataset_root, "train2017"), 
                                        os.path.join(dataset_root, "annotations/instances_train2017.json"),
                                        num_images=500)
    val_loader = get_coco_data_loader(os.path.join(dataset_root, "val2017"), 
                                      os.path.join(dataset_root, "annotations/instances_val2017.json"),
                                      num_images=100)
    
    for epoch in range(1):
        print(f"Epoch {epoch+1}")
        train(model, train_loader, optimizer, device)

    
    mAP, wada, iou = evaluate(model, val_loader, device)
    print(f"Evaluation Results: mAP={mAP:.4f}, WADA={wada:.4f}, IoU={iou:.4f}")

    torch.save(model.state_dict(), "faster_rcnn_coco2017.pth")

if __name__ == "__main__":
    main()


loading annotations into memory...
Done (t=20.40s)
creating index...
index created!
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
Epoch 1


Evaluation Loop: 100%|██████████| 25/25 [00:17<00:00,  1.41it/s]


Evaluation Results: mAP=0.4169, WADA=0.3700, IoU=0.3232
